In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
test_df = pd.read_csv ('Dataset/testset.csv')

data_test = test_df[['x_test','future']].to_numpy()

x_test_string = data_test[:,0]
y_test_string = data_test[:,1]
x_test = []
y_test = []
for i in x_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_test = [int(j) for j in i.split()]
    x_test.append(input_x_test)

for i in y_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_test = [int(j) for j in i.split()]
    y_test.append(input_y_test)
x_test = np.array(x_test)
y_test = np.array(y_test)
    


notes_ = notes_df.to_numpy()[:,1]
unique_notes = dict(enumerate(notes_.flatten(), 0))
# unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}


In [3]:
from MusicDataset import *
test_set = MusicDataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

In [4]:
import torch
from Models import Wavenet, LSTM

Net = torch.load('Trained_Model/model.pth', map_location=torch.device('cpu'))
print(Net)


Sequence(
  (embedding): Embedding(173, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (softmax): Softmax(dim=None)
  (linear3): Linear(in_features=128, out_features=173, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [ ]:
#import torch
#print(torch.__version__)

In [5]:
total_preds = 0
correct_preds = 0
future_preds = 8
for i, data in enumerate(testloader, 0):

    input , label = data
    cumm_output = torch.zeros(0,len(unique_notes)).to(device)
    cumm_label  = np.array([],dtype=int)
    for k in range(future_preds):
        output = Net(input.to(device),input.shape[0])
        cumm_output = torch.cat((cumm_output,output))
        cumm_label = np.concatenate((cumm_label,label[:,k]))
        next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label[:,k].to(device))
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
                                               for ind,j in enumerate(input)])[:,1:]) 
test_acc =  float(correct_preds)/float(total_preds) *100 
testreport ="Testing Accuracy : \n correct predictions  : {} \n total predictions : {} \n Testing Accuracy : {} \n ------------------------\n".format(correct_preds,total_preds,test_acc)
print(testreport)  

Testing Accuracy : 
 correct predictions  : 8004 
 total predictions : 64112 
 Testing Accuracy : 12.484402295982031 
 ------------------------



In [8]:
# Time to generate some Music!!!
import random
from Dataset import midi_helper 
for j in range(10):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device), input.shape[0])
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])
        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
    path = './Outputs/music'+str(j)+'.midi'
    midi_helper.convert_to_midi(tune)  # Remove the path argument


Taking the seed tune as follows:
[124  38 124 151  38 124  93 158 142 142 158  93 124  38  78 147 143  13
 143  13 143  13 143  13 112 143  13 155  27 141 120  78]
Taking the seed tune as follows:
[ 17 102  39   8 102 130 117   6 171 135  91 117 135  17  39   8 102 163
  97   4 108 125  39 107 146 108  45 163   6  29 146  45]
Taking the seed tune as follows:
[171 171 116  56  40 132 129 116 122  56 159   5 171   5 171   5 171  98
 171 159  50 109 109 109 109 109 109 109  73  50 109 109]
Taking the seed tune as follows:
[  5  50 129  29  50 113   9  21   9  97  68  62   9 115 125   9  21 125
   9  97  68 146   9 163  80  39  22  97  80   6  16  80]
Taking the seed tune as follows:
[ 45 149  29  12  45  29 149  45  29  12  45  45 149  29 130  12 117 108
 149   4 130  12  45  29   7 106  41  29 106   7  29   4]
Taking the seed tune as follows:
[ 81  43 115  43 115 128  81 128 115 128 115  81 115 115  43  81  43 115
  43 115  43  20 140 115 172  43 115 140  43  20  43 115]
Taking the seed 